In [3]:
import pandas as pd
import numpy as np

In [4]:
%%time
#twitter = pd.read_csv("preprocessed_data.csv")    #normal one used before
#twitter = pd.read_csv("sorted_preprocessed_data.csv")  #with lowest emotion scores
twitter = pd.read_csv("final_dataset.csv")
#twitter = pd.read_csv("greater_than_some_score.csv")

Wall time: 5.12 s


In [5]:
pd.set_option('display.max_rows', 1000)
twitter

,_score,tweet_id,text,identification,emotion,text_stemmed,Category
0,391,0x376b20,people who post add me on snapchat must be deh...,train,anticipation,peopl who post add me on snapchat must be dehy...,4
1,433,0x2d5350,brianklaas as we see trump is dangerous to fre...,train,sadness,brianklaa as we see trump is danger to freepre...,8
2,376,0x1cd5b0,now issa is stalking tasha 😂😂😂,train,fear,now issa is stalk tasha 😂😂😂,1
3,120,0x1d755c,riskshow thekevinallison thx for the best time...,train,joy,riskshow thekevinallison thx for the best time...,7
4,1021,0x2c91a8,still waiting on those supplies liscus,train,anticipation,still wait on those suppli liscu,4
...,...,...,...,...,...,...,...
1451386,94,0x321566,im so happy nowonder the name of this show hap...,train,joy,im so happi nowond the name of thi show happi ...,7
1451387,627,0x38959e,in every circumtance id like to be thankful to...,train,joy,in everi circumt id like to be thank to the al...,7
1451388,274,0x2cbca6,theres currently two girls walking around the ...,train,joy,there current two girl walk around the librari...,7
1451389,840,0x24faed,ah corporate life where you can date using ju...,train,joy,ah corpor life where you can date use just the...,7


In [6]:
#Dividing the dataframe into a train and a test sections
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(twitter.text,twitter.emotion,
                                                test_size=0.2, random_state = 42)

In [10]:
%%time
import nltk
from sklearn.feature_extraction.text import CountVectorizer

BOW = CountVectorizer(tokenizer=nltk.word_tokenize, max_features=25000)

BOW.fit(x_train)

Wall time: 2min 2s


CountVectorizer(max_features=25000,
                tokenizer=<function word_tokenize at 0x0000016146D6FD30>)

In [11]:
feature_names = BOW.get_feature_names()

In [12]:
"😕" in feature_names

True

In [13]:
x_train

680793     thank you for the follow growhackwin we post i...
815757     newsweek of course realdonaldtrump does becaus...
1124461    thehighfrontier spacelectures cosmiccarol ther...
352702     hey maga people kellyannepolls is your whiteho...
352523     a year later even my worst days look sunny in ...
                                 ...                        
259178               ibizaradio fincaam today is simply   it
1414414    thanks to my followers you make me  chaturbate...
131932     your mind will always believe what you tell it...
671155     it is a  day here happy humpday  for those in ...
121958     djspinall  albumsurely feel like making cover ...
Name: text, Length: 1161112, dtype: object

In [14]:
import keras

x_train = BOW.transform(x_train)
#y_train = y_train

x_test = BOW.transform(x_test)
#y_test = y_test

print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)
print('x_test.shape: ', x_test.shape)
print('y_test.shape: ', y_test.shape)

x_train.shape:  (1161112, 25000)
y_train.shape:  (1161112,)
x_test.shape:  (290279, 25000)
y_test.shape:  (290279,)


In [15]:
y_test

63955           disgust
204069     anticipation
1256026             joy
561398            trust
1094198           trust
               ...     
78967           disgust
130772              joy
760291              joy
482430             fear
1265785             joy
Name: emotion, Length: 290279, dtype: object

In [16]:
type(x_train)

scipy.sparse.csr.csr_matrix

In [17]:
y_train

680793            trust
815757          sadness
1124461        surprise
352702          sadness
352523            trust
               ...     
259178              joy
1414414             joy
131932              joy
671155     anticipation
121958     anticipation
Name: emotion, Length: 1161112, dtype: object

In [18]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    #return keras.utils.to_categorical(enc)
    return keras.utils.np_utils.to_categorical(enc)   #Allison/Moo said so, because of version

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)


check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 680793        trust
815757      sadness
1124461    surprise
352702      sadness
Name: emotion, dtype: object

y_train.shape:  (1161112,)
y_test.shape:  (290279,)


## After convert
y_train[0:4]:
 [[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (1161112, 8)
y_test.shape:  (290279, 8)


In [19]:
# I/O check
input_shape = x_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  25000
output_shape:  8


In [20]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 10000
X = model_input

# 1st hidden layer
X_W1 = Dense(units=32)(X)  # Original: 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=32)(H1)  # Original: 64
H2 = ReLU()(H1_W2)

# 3rd hidden layer   (CREATED BY ME)
H1_W4 = Dense(units=32)(H2)  # Original: 64
H4 = ReLU()(H1_W4)

# output layer
#Original output layer
# H2_W3 = Dense(units=output_shape)(H2)  # 8
# H3 = Softmax()(H2_W3)

H2_W3 = Dense(units=output_shape)(H4)  # 8
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25000)]           0         
                                                                 
 dense (Dense)               (None, 32)                800032    
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 re_lu_1 (ReLU)              (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 re_lu_2 (ReLU)              (None, 32)                0     

In [21]:
from keras.callbacks import CSVLogger
import tensorflow as tf

#csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 5 #25
batch_size = 64 #32, 100

es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max')
callbacks = [es] #Early Stopping

# training!
history = model.fit(x_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=callbacks,
                    validation_data = (x_test, y_test))
print('training finish')

Epoch 1/5


C:\Users\alexc\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/model/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


18143/18143 [==============================] - 116s 6ms/step - loss: 1.3042 - accuracy: 0.5290 - val_loss: 1.2572 - val_accuracy: 0.5469
Epoch 2/5
18143/18143 [==============================] - 118s 6ms/step - loss: 1.2003 - accuracy: 0.5666 - val_loss: 1.2487 - val_accuracy: 0.5515
training finish


In [22]:
## predict
pred_result = model.predict(x_test, batch_size=128) #128
pred_result[:5]

array([[0.46925405, 0.0264368 , 0.07121582, 0.01711301, 0.20772894,
        0.13972332, 0.02822841, 0.04029969],
       [0.00816352, 0.366828  , 0.02646521, 0.03951548, 0.2594771 ,
        0.04246772, 0.02323408, 0.23384887],
       [0.06588798, 0.03595962, 0.19296746, 0.01670147, 0.11860583,
        0.50190634, 0.02677083, 0.04120046],
       [0.00488782, 0.2859522 , 0.19171445, 0.03740311, 0.3037289 ,
        0.03958802, 0.08837623, 0.04834925],
       [0.02422241, 0.02394267, 0.08939138, 0.02971592, 0.1352106 ,
        0.59445167, 0.04045147, 0.06261387]], dtype=float32)

In [23]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['anger', 'anticipation', 'sadness', 'joy', 'sadness'], dtype=object)

In [24]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), pred_result), 4)))

#Best until now 0.5526

testing accuracy: 0.5515


### Predicting on test data

In [25]:
twitter_test_data = pd.read_csv("twitter_test_data.csv")

In [26]:
x_test_twitter = BOW.transform(twitter_test_data['text'])

pred_result_test_data = model.predict(x_test_twitter, batch_size=128)

print('x_test.shape: ', x_test_twitter.shape)
pred_result_test_data[:5]

x_test.shape:  (411972, 25000)


array([[6.8423120e-05, 5.6850082e-01, 2.1908390e-04, 2.7880049e-03,
        2.2437191e-01, 1.9427638e-03, 1.0160194e-03, 2.0109303e-01],
       [5.2352159e-05, 9.1652673e-01, 1.7295823e-04, 4.3415729e-04,
        1.2528725e-02, 1.5445192e-03, 3.0595323e-04, 6.8434618e-02],
       [4.3726298e-03, 2.8767979e-01, 2.6688341e-02, 1.1482280e-02,
        5.2606267e-01, 9.6089236e-02, 8.9859394e-03, 3.8639143e-02],
       [3.1773979e-04, 5.5826157e-01, 5.1279011e-04, 3.2611496e-03,
        2.7685979e-01, 1.1480423e-03, 7.9531065e-04, 1.5884362e-01],
       [5.0140442e-03, 1.4953013e-01, 1.3320454e-02, 3.0009821e-02,
        1.6605540e-01, 4.4021897e-02, 1.1938519e-02, 5.8010983e-01]],
      dtype=float32)

In [27]:
pred_result_test_data = label_decode(label_encoder, pred_result_test_data)
pred_result_test_data[:5]

array(['anticipation', 'anticipation', 'joy', 'anticipation', 'trust'],
      dtype=object)

In [28]:
upload_df = pd.DataFrame(columns = [["id","emotion"]])
upload_df["id"] = twitter_test_data["tweet_id"]
upload_df["emotion"] = pred_result_test_data
upload_df

,id,emotion
0,0x28b412,anticipation
1,0x2de201,anticipation
2,0x218443,joy
3,0x2939d5,anticipation
4,0x26289a,trust
...,...,...
411967,0x2913b4,joy
411968,0x2a980e,anticipation
411969,0x316b80,sadness
411970,0x29d0cb,anger


In [29]:
upload_df['emotion']
upload_df.emotion.nunique()

emotion    8
dtype: int64

In [30]:
upload_df.to_csv("./uploads/Keras_25k.csv",index=False)

## With Logistic Regression

Use this one with only the BOW and the x_train, y_train BOW transformed. (using the category column)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(x_train, y_train)
score = classifier.score(x_test, y_test)

print("Accuracy:", score)

## Another Keras Model Try (failed atm)

In [ ]:
from keras.models import Sequential
from keras import layers

input_dim = x_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',#'binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])
model.summary()

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

In [ ]:
model